In [45]:
# Define variables
n = 3
N = 2^n
X = var(['X{}'.format(i) for i in range(n)]) # Define the variables X_0, X-1,.....
u = var(['u{}'.format(i) for i in range(n)]) # Define the variables u_0, u-1,.....

# Function to get binary representation as a list of bits
def bits(i,n):
    return list(map(int, format(i,'0{}b'.format(n))))

def bits_reverse(i, n):
    bits_list = bits(i, n)
    return list(reversed(bits_list))

# Define the eq_tilde function
def eq_tilde(bits_i, u_vector):
    result=1
    for bit,u in zip(bits_i,u_vector):
        result *= (1-bit)*(1-u) + bit*u
    return result

# Coefficients of the polynomial
a = [var('a{}'.format(i)) for i in range(N)] # Coefficients a_0, a_1, ...., a_(N-1)

# MLE polynomial
f_tilde = sum(a[i]*eq_tilde(bits(i,n), X) for i in range(N))

s = str(f_tilde)

# 2. Eliminate any embedded newlines (if Sage wrapped long lines)
single = s.replace('\n', ' ')

# 3. Split on each ‘ + ’—this gives you each summand without losing its internal structure
terms = single.split(' + ')

# 4. Print each term on its own line, re-attaching the '+' to all but the last
for i, term in enumerate(terms):
    if i < len(terms) - 1:
        print(term + " +")
    else:
        print(term)
# Generate all combinations of (1-u[i]) and u[i] based on binary representation
def generate_c_vector(n, u):
    c_vector = []
    for i in range(2^n):  # Loop over all binary numbers from 0 to 2^n - 1
        binary = list(map(int, format(i, f'0{n}b')))  # Binary representation of i
        binary_reverse = list(reversed(binary))  # Reverse the binary representation
        product = 1
        for j, bit in enumerate(binary_reverse):
            if bit == 0:
                product *= (1 - u[j])  # Use (1 - u[j]) for 0
            else:
                product *= u[j]  # Use u[j] for 1
        c_vector.append(product)
    return c_vector

# Compute the c vector
c = generate_c_vector(n, u)

# Display the c vector
show(c)

# Create a 3D plot of f_tilde
# First, let's create a numerical version of f_tilde by substituting some values for coefficients
import numpy as np

# Set some random coefficients for visualization
np.random.seed(42)
coeffs = np.random.rand(8)  # 8 coefficients for n=3

# Create a numerical function
def f_tilde_numerical(x0, x1, x2):
    result = 0
    for i in range(8):
        bits_i = bits(i, 3)
        result += coeffs[i] * eq_tilde(bits_i, [x0, x1, x2])
    return result

# Create points for all evaluations
points = []
for i in range(8):
    bits_i = bits(i, 3)
    x, y, z = bits_i
    points.append((x, y, z, coeffs[i]))

# Create point plot with axes labels
point_plot = point3d([(p[0], p[1], p[2]) for p in points], size=20, axes_labels=['X0', 'X1', 'X2'])

# Create text labels with offset, only showing coordinate and a value
text_plots = []
offset = 0.22  # Spacing for clarity
fontsize = 16  # Large and readable

for i, p in enumerate(points):
    label = f"({p[0]},{p[1]},{p[2]})\n" + f"a{i}={p[3]:.2f}"
    # If x=1, place label to the right; else to the left
    if p[0] == 1:
        label_pos = (p[0] + offset, p[1], p[2])
    else:
        label_pos = (p[0] - offset, p[1], p[2])
    text_plots.append(text3d(label, label_pos, fontsize=fontsize))

# Combine all text labels into a single plot (no blue dots)
final_plot = sum(text_plots)
final_plot.show()

# --- Separate a-vector display section ---
a_vector_text = "a = [" + ", ".join([f"{c:.2f}" for c in coeffs]) + "]"
show(html(f"<pre style='font-size:14px'>{a_vector_text}</pre>"))

-(X0 - 1)*(X1 - 1)*(X2 - 1)*a0 +
(X0 - 1)*(X1 - 1)*X2*a1 +
(X0 - 1)*X1*(X2 - 1)*a2 - (X0 - 1)*X1*X2*a3 +
X0*(X1 - 1)*(X2 - 1)*a4 - X0*(X1 - 1)*X2*a5 - X0*X1*(X2 - 1)*a6 +
X0*X1*X2*a7


[-(u0 - 1)*(u1 - 1)*(u2 - 1),
 u0*(u1 - 1)*(u2 - 1),
 (u0 - 1)*u1*(u2 - 1),
 -u0*u1*(u2 - 1),
 (u0 - 1)*(u1 - 1)*u2,
 -u0*(u1 - 1)*u2,
 -(u0 - 1)*u1*u2,
 u0*u1*u2]

Graphics3d Object

<pre style='font-size:14px'>a = [0.37, 0.95, 0.73, 0.60, 0.16, 0.16, 0.06, 0.87]</pre>